In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt

from nltk.tokenize import TweetTokenizer
from scipy.special import softmax
from sklearn.metrics import classification_report
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv('preprocessed_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287188 entries, 0 to 287187
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   287187 non-null  float64
 1   createdAt            287188 non-null  object 
 2   fullName             287149 non-null  object 
 3   userName             287188 non-null  object 
 4   profileImage         287188 non-null  object 
 5   fullText             287188 non-null  object 
 6   replyTo              287188 non-null  int64  
 7   lang                 287188 non-null  object 
 8   quoteCount           287188 non-null  int64  
 9   retweetCount         287188 non-null  int64  
 10  replyCount           287188 non-null  int64  
 11  likeCount            287188 non-null  int64  
 12  viewCount            103504 non-null  float64
 13  sentimentLabel1      1790 non-null    float64
 14  sentimentLabel2      1201 non-null    float64
 15  sentimentLabel3  

# VADER



In [4]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to C:\Users\Krithika
[nltk_data]     JK\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
labeled_data = data[data['sentimentLabelFinal'].notnull()]
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 996 entries, 152428 to 153426
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   996 non-null    float64
 1   createdAt            996 non-null    object 
 2   fullName             996 non-null    object 
 3   userName             996 non-null    object 
 4   profileImage         996 non-null    object 
 5   fullText             996 non-null    object 
 6   replyTo              996 non-null    int64  
 7   lang                 996 non-null    object 
 8   quoteCount           996 non-null    int64  
 9   retweetCount         996 non-null    int64  
 10  replyCount           996 non-null    int64  
 11  likeCount            996 non-null    int64  
 12  viewCount            0 non-null      float64
 13  sentimentLabel1      996 non-null    float64
 14  sentimentLabel2      996 non-null    float64
 15  sentimentLabel3      996 non-nul

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report

train_data = data[['fullText', 'cleaned_tweet']]
validation_data = labeled_data[['fullText', 'cleaned_tweet', 'sentimentLabelFinal']]

# Initialize VADER SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment label
def get_sentiment_label(score):
    if score['compound'] >= 0.05:
        return 2  # positive
    elif score['compound'] <= -0.05:
        return 0  # negative
    else:
        return 1  # neutral


# Predict sentiment for validation data using VADER
predicted_sentiments_val = []

for text in validation_data['cleaned_tweet']:
    vader_score = sid.polarity_scores(text)
    predicted_sentiment = get_sentiment_label(vader_score)
    predicted_sentiments_val.append(predicted_sentiment)

# Save predicted sentiments for validation data to a DataFrame
predicted_sentiments_val_df = pd.DataFrame({'text': validation_data['cleaned_tweet'], 'sentiment': validation_data['sentimentLabelFinal'], 'predicted_sentiment': predicted_sentiments_val})

# Predict sentiment for training data using VADER
predicted_sentiments_train = []

for text in train_data['cleaned_tweet']:
    vader_score = sid.polarity_scores(text)
    predicted_sentiment = get_sentiment_label(vader_score)
    predicted_sentiments_train.append(predicted_sentiment)

# Save predicted sentiments for training data to a DataFrame
predicted_sentiments_train_df = pd.DataFrame({'fullText': train_data['fullText'], 'cleaned_tweet': train_data['cleaned_tweet'], 'predicted_sentiment': predicted_sentiments_train})

true_labels_val = validation_data['sentimentLabelFinal']
report_val = classification_report(true_labels_val, predicted_sentiments_val)
print("Classification Report:")
print(report_val)

#predicted_sentiments_train_df.to_csv("predicted_sentiments_val.csv", index=False)


Classification Report:
              precision    recall  f1-score   support

         0.0       0.56      0.52      0.54       355
         1.0       0.69      0.43      0.53       497
         2.0       0.30      0.74      0.43       144

    accuracy                           0.51       996
   macro avg       0.52      0.57      0.50       996
weighted avg       0.59      0.51      0.52       996



In [46]:
predicted_sentiments_val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1992 entries, 0 to 171005
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 1992 non-null   object 
 1   sentiment            1992 non-null   float64
 2   predicted_sentiment  1992 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 62.2+ KB


In [47]:
predicted_sentiments_train_df.head(5)

,fullText,cleaned_tweet,predicted_sentiment
0,@nytimes The curse on Musk iis working!\r\n🔥☔🐱🐱🤹💥,the curse on musk iis working :fire::umbrellaw...,0
1,@elonmusk and @joebiden walk into a Chinese ba...,and walk into a chinese bar elon makes money l...,2
2,Elon Musk helped create China’s electric vehic...,elon musk helped create chinas electric vehicl...,2
3,Lies.\r\n\r\nMusk faced even greater challenge...,lies musk faced even greater challenges from h...,1
4,@Sp4rksaflyin @nytimes These Electric Tesla ca...,these electric tesla cars cannot start in the ...,2


In [48]:
predicted_sentiments_train_df['predicted_sentiment'].value_counts()

2    112238
0    105200
1     86524
Name: predicted_sentiment, dtype: int64